In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(spark.sparkContext, 'be63ac50-4f9d-43f3-8f26-ad6572f65430', 'p-7bda336230f76ada26590bc6c4038045546d3287')
pc = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210531012322-0024
KERNEL_ID = d99b1e73-5674-48f7-9a3d-d089a64a120e


In [2]:
# Importando algumas bibliotecas
import sys
import shutil
import types
import pandas as pd
pd.options.display.max_columns = 999

from botocore.client import Config
import ibm_boto3

from datetime import datetime
from datetime import datetime, timedelta

In [3]:
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-8c64e377-89e3-424a-b706-72ea9ab5fd59',
    'IBM_API_KEY_ID': 'vNrUHeW9-R1HdOM2ld5PDpR96H0VdqryNzcmyVlNFne_',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'projetoviagrupo1-donotdelete-pr-q8ymlbqhihqlvv',
    'CES' : 'compraentregastatus.csv',
    'SL' : 'skulojista.csv',
    'CE' : 'compraentrega.csv',
    'C' : 'compra.csv',
    'CESKU' : 'compraentregasku.csv',
    'CFPG' : 'compraformapagamento.csv',
    'LOJ' : 'lojista.csv',
    'CAT': 'skucategoria.csv',
    'NPS' : 'visaolojistavendanps.csv',
    'FPG' : 'formapagamento.csv'
}

In [4]:
cgsClient = ibm_boto3.client(service_name='s3',
    ibm_api_key_id = credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [5]:
path = '/home/spark/shared/tmp/'
try:
    os.makedirs(path)
except OSError:
    shutil.rmtree(path)
    print ("Deletado diretório: %s" % path)
    print ("Diretório criado com sucesso: %s" % path)
    os.makedirs(path)
else:
    print ("Diretório criado com sucesso: %s" % path)

Deletado diretório: /home/spark/shared/tmp/
Diretório criado com sucesso: /home/spark/shared/tmp/


In [6]:
def upload_file_cos(cos, local_file_name, credentials,key):  
    
    try:
        res=cos.upload_file(Filename=local_file_name+key, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(key+' - File Uploaded -'+ (datetime.now() - timedelta(hours=3)).strftime("%d/%m/%Y %H:%M:%S"))

In [7]:
def download_file_cos(cos, local_file_name, credential): 
    
    for i in credential:
        if list(credential).index(i) > 4:

            '''
            Wrapper function to download a file from cloud object storage using the
            credential dict provided and loading it into memory
            '''
            try:
                res=cos.download_file(Bucket=credential['BUCKET'], Key=credential[i], Filename=local_file_name +credential[i])
            except Exception as e:
                print(credential[i] + ' - Exception', e)
            else:
                print(credential[i] + ' - File Downloaded')
    
    print("Done. -"+(datetime.now() - timedelta(hours=3)).strftime("%d/%m/%Y %H:%M:%S"))

In [8]:
download_file_cos(cgsClient, path, credentials_1)

compraentregastatus.csv - File Downloaded
skulojista.csv - File Downloaded
compraentrega.csv - File Downloaded
compra.csv - File Downloaded
compraentregasku.csv - File Downloaded
compraformapagamento.csv - File Downloaded
lojista.csv - File Downloaded
skucategoria.csv - File Downloaded
visaolojistavendanps.csv - File Downloaded
formapagamento.csv - File Downloaded
Done. -30/05/2021 22:24:11


<h1>ETL</h1>

In [9]:
spark.read.option("header",'true').csv("/home/spark/shared/tmp/compraentregastatus.csv").createOrReplaceTempView("compraentregastatuslog")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/skulojista.csv").createOrReplaceTempView("skulojista")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/compraentrega.csv").createOrReplaceTempView("compraentrega")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/compra.csv").createOrReplaceTempView("compra")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/compraentregasku.csv").createOrReplaceTempView("compraentregasku")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/compraformapagamento.csv").createOrReplaceTempView("compraformapagamento")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/lojista.csv").createOrReplaceTempView("lojista")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/skucategoria.csv").createOrReplaceTempView("categoria")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/visaolojistavendanps.csv").createOrReplaceTempView("visaolojistavendanps")
spark.read.option("header",'true').csv("/home/spark/shared/tmp/formapagamento.csv").createOrReplaceTempView("formapagamento")

In [10]:
pgto = spark.sql('''
SELECT 
IDCOMPRA,
IDBANDEIRA,

Boleto,
Cartao_Presente,
Credito,
Debito,
Outro as Outro_Meio_Pagamento,
Vale,
Total_Pagamentos_Insucesso,
Total_Pagamentos_Sucesso

FROM
(SELECT 
ROW_NUMBER() OVER (
    PARTITION BY IDCOMPRA, IDBANDEIRA
    ORDER BY idcompraformapagamentostatus ASC
) as LINHA,
IDCOMPRA,
IDBANDEIRA,
idcompraformapagamentostatus,

COALESCE(Boleto,0) AS Boleto,
COALESCE(`Cartão Presente`,0) AS Cartao_Presente,
COALESCE(Credito,0) AS Credito,
COALESCE(Debito,0) AS Debito,
COALESCE(Outro,0) AS Outro,
COALESCE(Vale,0) AS Vale,

COALESCE(Total_Pagamentos_Insucesso,0) AS Total_Pagamentos_Insucesso,
COALESCE(Total_Pagamentos_Sucesso,0) AS Total_Pagamentos_Sucesso

FROM (
select
CE.IDCOMPRA,
CE.IDBANDEIRA,
CASE 
  WHEN FPG.nome = 'Pix' AND FPG.tipo = 'QrCode' THEN FPG.nome
  WHEN FPG.tipo in ('Cartao de Debito', 'Loja Debito') THEN 'Debito'
  WHEN FPG.tipo = 'Loja Credito' or FPG.tipo = 'Credito' THEN 'Credito'
  WHEN FPG.tipo IN ('Boleto Prazo B2B','Boleto B2B') THEN 'Boleto B2B'
  WHEN FPG.tipo IN ('Boleto', 'Carne', 'Cartão Presente', 'Credito', 'Debito', 'Loja Dinheiro','QrCode', 'Vale') THEN FPG.tipo
  ELSE 'Outro'
  END AS Tipo_Pagamento,
CFPG.idcompraformapagamentostatus,
count(DISTINCT CFPG.idcompraformapagamento) as Quantidade_Tentativas_Pagamentos,

sum(count(distinct (case when CFPG.idcompraformapagamentostatus != 2 then CFPG.idcompraformapagamento else null end))) OVER (
    PARTITION BY CE.IDCOMPRA, CE.IDBANDEIRA 
    ORDER BY CE.IDCOMPRA ASC
) as Total_Pagamentos_Insucesso,

sum(count(distinct (case when CFPG.idcompraformapagamentostatus = 2 then CFPG.idcompraformapagamento else null end))) OVER (
    PARTITION BY CE.IDCOMPRA, CE.IDBANDEIRA 
    ORDER BY CE.IDCOMPRA ASC
) as Total_Pagamentos_Sucesso

from compraentrega CE

LEFT JOIN compraformapagamento CFPG
on CE.IDCOMPRA = CFPG.idcompra and CE.IDBANDEIRA = CFPG.IDBANDEIRA

LEFT JOIN FORMAPAGAMENTO FPG
ON CFPG.IDFORMAPAGAMENTO = FPG.IDFORMAPAGAMENTO AND CFPG.IDBANDEIRA = FPG.IDBANDEIRA

where CE.origem = 'LJ'
group by 1,2,3,4)
PIVOT (
sum(Quantidade_Tentativas_Pagamentos)
FOR Tipo_Pagamento in ('Boleto', 'Cartão Presente', 'Credito', 'Debito', 'Outro', 'Vale')))
WHERE LINHA = 1
''')
pgto.createOrReplaceTempView('pivot_pgto')

In [11]:
carrinho = spark.sql('''select 
ROW_NUMBER() OVER (
    PARTITION BY ce.idcompra, ce.IDBANDEIRA 
    ORDER BY coalesce(ROUND((CESKU.valorvendaunidade + CESKU.valorfretecomdesconto),2),val.Valor+val.Frete) desc
) as Item_Carrinho,

ce.idcompra,
ce.IDBANDEIRA,
c.flagaprovado,
C.idcanalvenda,
ce.idfreteentregatipo,
ce.idcompraentrega,
ce.idcompraentregastatus as status,
dev.Devolucao,

c.Parceiro,
c.Campanha,
  
CE.idlojista,
ce.flagenviadomarketplace,

c.data,
date_format(C.data, 'HH:mm:ss') as Hora_Pedido,
date(C.data) as Dia_Pedido,
date_format(C.data,'yyyy-MM') as Mes_pedido,

CASE
    WHEN date(ce.dataentrega) > date(ce.dataprevisao) THEN 1
    ELSE 0
    END AS Flag_Atraso,
    
DATEDIFF(date(ce.dataprevisao),date(ce.dataentrega)) as Tempo_atraso,

ce.dataentrega,
ce.dataprevisao,

CESKU.idsku,
coalesce(ROUND(CESKU.valorvendaunidade,2),val.Valor) as Valor,
coalesce(ROUND(CESKU.valorfretecomdesconto,2),val.Frete) as Frete,
coalesce(ROUND((CESKU.valorvendaunidade + CESKU.valorfretecomdesconto),2),val.Valor+val.Frete) AS Valor_Com_Frete,

CAT.idsetor AS IDCATEGORIA,
CAT.nomesetor AS CATEGORIA,
CAT.iddepartamento AS IDDEPARTAMENTO,
CAT.nomedepartamento AS DEPARTAMENTO,

pg.Boleto,
pg.Cartao_Presente,
pg.Credito,
pg.Debito,
pg.Outro_Meio_Pagamento,
pg.Vale,
pg.Total_Pagamentos_Insucesso,
pg.Total_Pagamentos_Sucesso

from compraentrega CE

LEFT JOIN compra C
ON CE.IDCOMPRA = C.IDCOMPRA AND CE.IDBANDEIRA = C.IDBANDEIRA

LEFT JOIN (select
idcompra,
idbandeira,
max(case when origem = 'TD' then 1 else 0 end) as Devolucao
from compraentrega
group by 1,2) DEV
ON CE.IDCOMPRA = DEV.IDCOMPRA AND CE.IDBANDEIRA = DEV.IDBANDEIRA

LEFT JOIN lojista LJ
ON CE.idlojista = LJ.idlojista AND CE.IDBANDEIRA = LJ.IDBANDEIRA

LEFT JOIN compraentregasku CESKU
on CESKU.IDBANDEIRA = CE.IDBANDEIRA AND CESKU.idcompraentrega = ce.idcompraentrega AND CESKU.idcompraentregaskupai IS NULL

LEFT JOIN categoria CAT
on CESKU.idsku = CAT.idsku

LEFT JOIN pivot_pgto pg
on ce.idcompra = pg.idcompra and ce.IDBANDEIRA = pg.IDBANDEIRA

LEFT JOIN (select 
idsku,
idbandeira,
round(avg(valorvendaunidade),2) as Valor,
round(avg(valorfretecomdesconto),2) as Frete

from compraentregasku
where idcompraentregaskupai IS NULL
group by 1,2) val
on CESKU.idsku = val.idsku and CESKU.idbandeira = val.idbandeira

WHERE CE.idfreteentregatipo NOT IN ('17','19','20')
AND CE.origem = 'LJ'
''')
carrinho.createOrReplaceTempView('carrinho')

In [12]:
carrinho.toPandas().to_csv('/home/spark/shared/tmp/carrinho.csv', index=False)
upload_file_cos(cgsClient, path,credentials_1,'carrinho.csv')

carrinho.csv - File Uploaded -30/05/2021 22:29:48


In [13]:
preco_lojista = spark.sql('''
select 
LJ.idlojista,
LJ.idbandeira,
SKL.idsku,
CAT.idsetor AS IDCATEGORIA,
CAT.nomesetor AS CATEGORIA,
CAT.iddepartamento AS IDDEPARTAMENTO,
CAT.nomedepartamento AS DEPARTAMENTO,
SKL.precoanterior,
SKL.precovenda AS precoatual,
SKL.dataultimaatualizacao as DataAtualizacao

from lojista LJ

LEFT JOIN skulojista SKL
ON LJ.idlojista = SKL.idlojista AND LJ.idbandeira = SKL.idbandeira

LEFT JOIN CATEGORIA CAT
ON SKL.idsku = CAT.idsku
''')

In [14]:
preco_lojista.toPandas().to_csv('/home/spark/shared/tmp/preco_lojista.csv', index=False)
upload_file_cos(cgsClient, path,credentials_1,'preco_lojista.csv')

preco_lojista.csv - File Uploaded -30/05/2021 22:31:12


In [15]:
preco_lojista.toPandas().to_json('/home/spark/shared/tmp/preco_lojista.json',orient="records")
upload_file_cos(cgsClient, path,credentials_1,'preco_lojista.json')

preco_lojista.json - File Uploaded -30/05/2021 22:31:44


In [16]:
dia_nps = spark.sql('''
select 
year(DtAprovacaoPedidoSite) as ano,
month(DtAprovacaoPedidoSite) as mes,
day(DtAprovacaoPedidoSite) as dia,
idlojista,
idbandeira AS Bandeira,
NmClusterLojista as ClusterGMV,
sum(QtResposta) as QtResposta,
sum(QtRespostaDetratora) as QtRespostaDetratora,
sum(QtRespostaPromotora) as QtRespostaPromotora,
((sum(QtRespostaPromotora) - sum(QtRespostaDetratora)) / sum(QtResposta)) as `%NPS`

from visaolojistavendanps
group by 1,2,3,4,5,6
''')
dia_nps.createOrReplaceTempView('dia_nps')

In [17]:
dia_nps.toPandas().to_csv('/home/spark/shared/tmp/dia_nps.csv', index=False)
upload_file_cos(cgsClient, path,credentials_1,'dia_nps.csv')

dia_nps.csv - File Uploaded -30/05/2021 22:31:51


In [18]:
compilado_vendas = spark.sql('''
select
T1.*,
coalesce(nps.QtResposta,0) as QtResposta,
coalesce(nps.QtRespostaDetratora,0) as QtRespostaDetratora,
coalesce(nps.QtRespostaPromotora,0) as QtRespostaPromotora,
coalesce(nps.`%NPS`,0) as `%NPS`

from
(select
year(dia_pedido) as ano,
month(dia_pedido) as mes,
day(dia_pedido) as dia,
idlojista,
IDBANDEIRA,
idsku,
IDCATEGORIA,
CATEGORIA,
IDDEPARTAMENTO,
DEPARTAMENTO,
sum(Valor) as Valor,
sum(Frete) as Frete,
sum(valor_com_frete) as Valor_Total,
count(distinct idcompra) as pedidos,
count(Item_Carrinho) itens,
count(distinct idcompraentrega) as ENVIOS,
sum(Devolucao) as Devolucoes,
count(distinct (case when Flag_Atraso = 1 THEN idcompraentrega ELSE null END)) as atrasos,
SUM(case when Flag_Atraso = 1 THEN (Tempo_atraso*(-1)) ELSE 0 END)/ count(distinct (case when Flag_Atraso = 1 THEN idcompraentrega ELSE null END)) as media_atraso,
SUM(case when UPPER(status) = 'ENT' THEN 1 ELSE 0 END) as itens_entregues,
SUM(case when UPPER(status) like '%CAN%' THEN 1 ELSE 0 END) as itens_cancelados,
SUM(case when UPPER(status) not like '%CAN%' and UPPER(status) != 'ENT' THEN 1 ELSE 0 END) as itens_fluxo

from carrinho
group by 1,2,3,4,5,6,7,8,9,10) T1

LEFT JOIN dia_nps nps
on T1.ano = nps.ano and T1.mes = nps.mes and T1.dia = nps.dia and T1.idlojista = nps.idlojista and  T1.idbandeira = nps.Bandeira
''')

In [19]:
compilado_vendas.toPandas().to_csv('/home/spark/shared/tmp/compilado_vendas.csv', index=False)
upload_file_cos(cgsClient, path,credentials_1,'compilado_vendas.csv')

compilado_vendas.csv - File Uploaded -30/05/2021 22:36:05


In [20]:
compilado_vendas.toPandas().to_json('/home/spark/shared/tmp/compilado_vendas.json',orient="records")
upload_file_cos(cgsClient, path,credentials_1,'compilado_vendas.json')

compilado_vendas.json - File Uploaded -30/05/2021 22:37:06


In [21]:
# compilado_lojista = spark.sql('''
# select
# year(dia_pedido) as ano,
# month(dia_pedido) as mes,
# day(dia_pedido) as dia,
# idlojista,
# IDBANDEIRA,
# sum(Valor) as Valor,
# sum(Frete) as Frete,
# sum(valor_com_frete) as Valor_Total,
# count(distinct idcompra) as pedidos,
# count(Item_Carrinho) itens,
# count(distinct idcompraentrega) as ENVIOS,
# count(distinct (case when Flag_Atraso = 1 THEN idcompraentrega ELSE null END)) as atrasos,
# SUM(case when Flag_Atraso = 1 THEN (Tempo_atraso*(-1)) ELSE 0 END)/ count(distinct (case when Flag_Atraso = 1 THEN idcompraentrega ELSE null END)) as media_atraso,
# SUM(case when UPPER(status) = 'ENT' THEN 1 ELSE 0 END) as itens_entregues,
# SUM(case when UPPER(status) like '%CAN%' THEN 1 ELSE 0 END) as itens_cancelados,
# SUM(case when UPPER(status) not like '%CAN%' and UPPER(status) != 'ENT' THEN 1 ELSE 0 END) as itens_fluxo

# from carrinho
# group by 1,2,3,4,5
# ''')

In [22]:
# top_sku = spark.sql('''
# select
# idlojista,
# IDBANDEIRA,
# idsku,
# IDCATEGORIA,
# CATEGORIA,
# IDDEPARTAMENTO,
# DEPARTAMENTO,
# sum(Valor) as Valor,
# sum(Frete) as Frete,
# sum(Valor_Total) as Valor_Total,
# sum(pedidos) as pedidos,
# sum(itens) itens,
# sum(ENVIOS) as ENVIOS,
# sum(atrasos) as atrasos,
# SUM(media_atraso)/sum(atrasos) as media_atraso,
# SUM(itens_entregues) as itens_entregues,
# SUM(itens_cancelados) as itens_cancelados,
# SUM(itens_fluxo) as itens_fluxo

# from vendas
# group by 1,2,3,4,5,6,7
# ''')

In [23]:
# top_categoria = spark.sql('''
# select
# idlojista,
# IDBANDEIRA,
# IDCATEGORIA,
# CATEGORIA,
# sum(Valor) as Valor,
# sum(Frete) as Frete,
# sum(Valor_Total) as Valor_Total,
# sum(pedidos) as pedidos,
# sum(itens) itens,
# sum(ENVIOS) as ENVIOS,
# sum(atrasos) as atrasos,
# SUM(media_atraso)/sum(atrasos) as media_atraso,
# SUM(itens_entregues) as itens_entregues,
# SUM(itens_cancelados) as itens_cancelados,
# SUM(itens_fluxo) as itens_fluxo

# from vendas
# group by 1,2,3,4
# ''')

In [24]:
# top_departamento = spark.sql('''
# select
# idlojista,
# IDBANDEIRA,
# IDDEPARTAMENTO,
# DEPARTAMENTO,
# sum(Valor) as Valor,
# sum(Frete) as Frete,
# sum(Valor_Total) as Valor_Total,
# sum(pedidos) as pedidos,
# sum(itens) itens,
# sum(ENVIOS) as ENVIOS,
# sum(atrasos) as atrasos,
# SUM(media_atraso)/sum(atrasos) as media_atraso,
# SUM(itens_entregues) as itens_entregues,
# SUM(itens_cancelados) as itens_cancelados,
# SUM(itens_fluxo) as itens_fluxo

# from vendas
# group by 1,2,3,4
# order by 9 desc
# ''')